### сглаживание

In [10]:
import pandas as pd
import numpy as np
simdoc_path = './simdoc.txt'
similar_queries = pd.read_csv(oldsimdoc_path, sep=' |\t', header=None)

<ipython-input-10-0dd97693b321>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  similar_queries = pd.read_csv(oldsimdoc_path, sep=' |\t', header=None)


In [12]:
from tqdm import notebook
q_to_sim_queries = dict()
for row in notebook.tqdm(similar_queries.values):
    current_query = row[0]
    similar_to_current = row[1:]
    q_to_sim_queries[current_query] = similar_to_current

In [15]:
df_path = './tr_hadoop_features.tsv'
df = pd.read_csv(df_path, sep = '\t', header = None)


In [1]:
feature_names = ['qhost_dbn_shows', 'qhost_dbn_a','qhost_CTR','qhost_dbn_s','qhost_dbn_last_click','qhost_clicks', 'qhost_shows', 'qhost_avg_pos','qhost_dbn_clicks']

In [21]:
smoothed_df = df.copy()

In [22]:
queries = df['QueryId'].values
hosts = df['HostId'].values
added_counter = 0
current_df = df

In [59]:
added_counter = 0
current_df = df

for current_row_idx in notebook.tqdm(range(current_df.shape[0])): 

    current_query = queries[current_row_idx]
    similar_queries = q_to_sim_queries[current_query] 
    current_host = hosts[current_row_idx]

    for current_feature_name in feature_names: 
        if np.isnan(current_df.iloc[current_row_idx][current_feature_name]):
            smooth_values = []
            for sim_q in similar_queries: 
                candidate_df = current_df[(current_df['QueryId'] == sim_q) & (current_df['HostId'] == current_host)]
           
                if candidate_df.shape[0] != 0:
                    nonan_candidate_df = candidate_df[~(np.isnan(candidate_df[current_feature_name]))]
                    if nonan_candidate_df.shape[0] != 0:
                        curr_smooth_value = nonan_candidate_df[current_feature_name].mean()
                        smooth_values.append(curr_smooth_value)
            count = len(smooth_values)
            if count != 0:
                weights = np.arange(count, 0, -1)
                smooth_final = np.dot(smooth_values, weights)
                smooth_final /= np.cumsum(weights)[-1]
                smoothed_df.loc[current_row_idx, current_feature_name] = smooth_final
                added_counter += 1

        else:
            pass

In [67]:
smoothed_df.to_csv('./smoothed_hadoop_features_tr', index=None)

In [1]:
df_path = './tt_hadoop_features.tsv'
df = pd.read_csv(df_path, sep = '\t', header = None)


In [2]:
queries = df['QueryId'].values
hosts = df['HostId'].values
added_counter = 0
current_df = df


In [3]:
added_counter = 0
current_df = df

for current_row_idx in notebook.tqdm(range(current_df.shape[0])): 

    current_query = queries[current_row_idx]
    similar_queries = q_to_sim_queries[current_query] 
    current_host = hosts[current_row_idx]

    for current_feature_name in feature_names: 
        if np.isnan(current_df.iloc[current_row_idx][current_feature_name]):
            smooth_values = []
            for sim_q in similar_queries: 
                candidate_df = current_df[(current_df['QueryId'] == sim_q) & (current_df['HostId'] == current_host)]
           
                if candidate_df.shape[0] != 0:
                    nonan_candidate_df = candidate_df[~(np.isnan(candidate_df[current_feature_name]))]
                    if nonan_candidate_df.shape[0] != 0:
                        curr_smooth_value = nonan_candidate_df[current_feature_name].mean()
                        smooth_values.append(curr_smooth_value)
            count = len(smooth_values)
            if count != 0:
                weights = np.arange(count, 0, -1)
                smooth_final = np.dot(smooth_values, weights)
                smooth_final /= np.cumsum(weights)[-1]
                smoothed_df.loc[current_row_idx, current_feature_name] = smooth_final
                added_counter += 1

        else:
            pass

In [4]:
smoothed_df.to_csv('./smoothed_hadoop_features_tt', index=None)